# ELMo: Contextual language embedding

## by Josh Taylor

Note that you will need to use the non-GPU accelerated run-time on this notebook due to the large memory useage of the ELMo model.

https://towardsdatascience.com/elmo-contextual-language-embedding-335de2268604

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/learning-stack/Colab-ML-Playbook/blob/master/NLP/ELMo-Contextual%20language%20embedding/Elmo_contextual_embeddings.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/learning-stack/Colab-ML-Playbook/blob/master/NLP/ELMo-Contextual%20language%20embedding/Elmo_contextual_embeddings.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This article will explore the latest in natural language modelling; deep contextualised word embeddings. The focus is more practical than theoretical with a worked example of how you can use the state-of-the-art ELMo model to review sentence similarity in a given document as well as creating a simple semantic search engine.

**The importance of context in NLP**

As we know, language is complex. Context can completely change the meaning of the individual words in a sentence. For example:

He kicked the bucket.
I have yet to cross-off all the items on my bucket list.
The bucket was filled with water.
In these sentences, whilst the word ‘bucket’ is always the same, it’s meaning is very different.


Words can have different meanings depending on context
Whilst we can easily decipher these complexities in language, creating a model which can understand the different nuances of the meaning of words given the surrounding text is difficult.

It is for this reason that traditional word embeddings (word2vec, GloVe, fastText) fall short. They only have one representation per word, therefore they cannot capture how the meaning of each word can change based on surrounding context.

**Introducing ELMo; Deep Contextualised Word Representations**

Enter ELMo. Developed in 2018 by AllenNLP, it goes beyond traditional embedding techniques. It uses a deep, bi-directional LSTM model to create word representations.

Rather than a dictionary of words and their corresponding vectors, ELMo analyses words within the context that they are used. It is also character based, allowing the model to form representations of out-of-vocabulary words.

This therefore means that the way ELMo is used is quite different to word2vec or fastText. Rather than having a dictionary ‘look-up’ of words and their corresponding vectors, ELMo instead creates vectors on-the-fly by passing text through the deep learning model.

## Imports:

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import preprocessing

!python -m spacy download en_core_web_md #you will need to install this on first load
import spacy
from spacy.lang.en import English
from spacy import displacy


    100% |████████████████████████████████| 120.9MB 47.3MB/s 
  Running setup.py install for en-core-web-md ... - \ | / - \ | / - done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



**1. Get the text data, clean and tokenize**

It is amazing how simple this is to do using Python string functions and spaCy. Here we do some basic text cleaning by:

a) removing line breaks, tabs and excess whitespace as well as the mysterious ‘xa0’ character;

b) splitting the text into sentences using spaCy’s ‘.sents’ iterator.

ELMo can receive either a list of sentence strings or a list of lists (sentences and words). Here we have gone for the former. We know that ELMo is character based, therefore tokenizing words should not have any impact on performance.

In [0]:
nlp = spacy.load('en_core_web_md')
from IPython.display import HTML
import logging
logging.getLogger('tensorflow').disabled = True #OPTIONAL - to disable outputs from Tensorflow

## Get the data 

The below downloads a Pandas Dataframe which is publically hosted on Google Drive (this should therefore work for anyone)

In [3]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


file_id = '1M_XljfV5t_nGjvhyfTPO9n2nfOweMwYx'
destination = 'temp'
download_file_from_google_drive(file_id, destination)

combined = pd.read_pickle('temp')

combined.head()

,Company,URL,Industry,HQ,Also Covers Companies,UK Modern Slavery Act,California Transparency in Supply Chains Act,Period Covered,text,pdf,error,FT_tfidf
0,118 118 Money,https://www.118118money.com/anti-slavery-state...,Consumer Finance,United Kingdom,NaN,True,False,2016-2017,Introduction\n\nThis statement is made pursuan...,0,0,"[-0.09019677307999371, 0.23825215930123844, 0...."
1,1Spatial Plc,https://1spatial.com/who-we-are/legal/modern-s...,Software,United Kingdom,NaN,True,False,2017,While 1Spatial’s turnover is below £36m and th...,0,0,"[-0.5010607985753625, 0.42660413175930045, -0...."
2,20/20 Projects,http://www.2020projects.co.uk/modernslaverypolicy,Commercial Services & Supplies,United Kingdom,NaN,True,False,2015-2016,html error,0,1,"[0.9405487179756165, 0.40332984924316406, 0.70..."
3,2M Holdings Ltd,https://www.2m-holdings.com/2m-holdings-ltd-mo...,Distributors,United Kingdom,NaN,True,False,2015-2016,Modern slavery is a crime resulting in an abho...,0,0,"[-0.390252637821283, 0.488747594191321, -0.238..."
4,3i Group plc,https://www.3i.com/media/3815/modern-slavery-s...,Capital Markets,United Kingdom,NaN,True,False,2017-2018,pdf error tika,1,1,"[1.0879868624172204, 0.44540999591903685, 0.80..."


## Create sentence embeddings

**2. Get the ELMo model using TensorFlow Hub:**

If you have not yet come across TensorFlow Hub, it is a massive time saver in serving-up a large number of pre-trained models for use in TensorFlow. Luckily for us, one of these models is ELMo. We can load in a fully trained model in just two few lines of code. How satisfying…

In [0]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

In [5]:
combined.loc[combined.Company.str.contains("Asos")]

,Company,URL,Industry,HQ,Also Covers Companies,UK Modern Slavery Act,California Transparency in Supply Chains Act,Period Covered,text,pdf,error,FT_tfidf
494,Asos plc,https://www.asosplc.com/~/media/Files/A/Asos-V...,Internet & Direct Marketing Retail,United Kingdom,NaN,True,False,2016-2018,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1,0,"[-0.8573993510860287, 0.11926992131730585, -0...."
495,Asos plc,https://www.asosplc.com/~/media/Files/A/ASOS/r...,Internet & Direct Marketing Retail,United Kingdom,NaN,True,False,2015-2016,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1,0,"[-0.6513810935113411, 0.04600498146602333, 0.0..."


In [6]:
text = combined.iloc[494].text
import re

#text represents our raw text document

text = text.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ')
text = ' '.join(text.split())
doc = nlp(text)

sentences = []
for i in doc.sents:
  if len(i)>1:
    sentences.append(i.string.strip())
    
len(sentences)

389

In [7]:
sentences[0:10]

['modern slavery statement september 2016 - march 2018',
 'facts & figures - 31 august 2017 slavery, servitude, forced labour, bonded labour, and human trafficking are issues of increasing global concern, affecting all sectors, regions and economies.',
 'modern slavery is fundamentally unacceptable within our business and supply chain, and combatting it is an important element of our overall approach to business and human rights.',
 'asos is committed to respecting, protecting and championing the human rights of all those who come into contact with our operations, including employees, supply chain workers, customers and local communities.',
 'we accept our responsibility to support transparency; to find and resolve problems, to regularly review our business practices, and to collaborate with others to protect the rights of workers, particularly those who are most vulnerable to abuses such as modern slavery.',
 'this statement has been published in accordance with the modern slavery act

To then use this model in anger we just need a few more lines of code to point it in the direction of our text document and create sentence vectors:

In [0]:
# This tells the model to run through the 'sentences' list and return the default output (1024 dimension sentence vectors).
embeddings = embed(
    sentences,
    signature="default",
    as_dict=True)["default"]

In [9]:
#Start a session and run ELMo to return the embeddings in variable x

%%time
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  x = sess.run(embeddings)

CPU times: user 3min 30s, sys: 17.1 s, total: 3min 47s
Wall time: 1min 56s


## Visualize the sentences using PCA and TSNE

**3. Use visualisation to sense-check outputs**

It is amazing how often visualisation is overlooked as a way of gaining greater understanding of data. Pictures speak a thousand words and we are going to create a chart of a thousand words to prove this point (actually it is 8,511 words).

Here we will use PCA and t-SNE to reduce the 1,024 dimensions which are output from ELMo down to 2 so that we can review the outputs from the model. I have included further reading on how this is achieved at the end of the article if you want to find out more.

In [0]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
y = pca.fit_transform(x)

from sklearn.manifold import TSNE

y = TSNE(n_components=2).fit_transform(y)

Using the amazing Plotly library, we can create a beautiful, interactive plot in no time at all. The below code shows how to render the results of our dimensionality reduction and join this back up to the sentence text. Colour has also been added based on the sentence length. As we are using Colab, the last line of code downloads the HTML file. This can be found below:

In [11]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)


data = [
    go.Scatter(
        x=[i[0] for i in y],
        y=[i[1] for i in y],
        mode='markers',
        text=[i for i in sentences],
    marker=dict(
        size=16,
        color = [len(i) for i in sentences], #set color equal to a variable
        opacity= 0.8,
        colorscale='Viridis',
        showscale=False
    )
    )
]
layout = go.Layout()
layout = dict(
              yaxis = dict(zeroline = False),
              xaxis = dict(zeroline = False)
             )
fig = go.Figure(data=data, layout=layout)
file = plot(fig, filename='Sentence encode.html')

from google.colab import files
files.download('Sentence encode.html') 

## Create a semantic search engine:

In [16]:
#@title Sementic search
#@markdown Enter a set of words to find matching sentences. 'results_returned' can beused to modify the number of matching sentences retured. To view the code behind this cell, use the menu in the top right to unhide...
search_string = "modern slave" #@param {type:"string"}
results_returned = "2" #@param [1, 2, 3]

from sklearn.metrics.pairwise import cosine_similarity


embeddings2 = embed(
    [search_string],
    signature="default",
    as_dict=True)["default"]

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  search_vect = sess.run(embeddings2)
  

cosine_similarities = pd.Series(cosine_similarity(search_vect, x).flatten())
output =""
for i,j in cosine_similarities.nlargest(int(results_returned)).iteritems():
  output +='<p style="font-family:verdana; font-size:110%;"> '
  for i in sentences[i].split():
    if i.lower() in search_string:
      output += " <b>"+str(i)+"</b>"
    else:
      output += " "+str(i)
  output += "</p><hr>"
    
output = '<h3>Results:</h3>'+output
display(HTML(output))
#   print(sentences[i])
#   print('\n')
